In [55]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import Imputer
from sklearn.impute import SimpleImputer


df_train = pd.read_csv('data/train.csv')
df_test = pd.read_csv('data/test.csv')

In [56]:
# Agrupaciones

# Zona Provincias
provincias_sur = ['Yucatán','Quintana Roo','Campeche','Tabasco', 'Veracruz', 'Oaxaca', 'Chiapas', 'Guerrero', 'Michoacán']
provincias_centro = ['Tlaxcala','Morelos','Puebla','Hidalgo','Aguascalientes', 'Distrito Federal', 'Jalisco', 'Edo. de México','Querétaro', 'Colima', 'Nayarit', 'Aguascalientes', 'Guanajuato', 'Hidalgo']
provincias_norte = ['Sonora','San luis Potosí','Baja California Norte', 'Baja California Sur', 'Chihuahua', 'Durango', 'Coahuila', 'Sinaloa', 'Nuevo León',  'Tamaulipas']

prov_por_precio = {'Distrito Federal':0, 'Edo. de México':1, 'Nuevo León':2, 'Guerrero':3, 'Quintana Roo':4, 'Jalisco':5, 'Tabasco':6, 'Puebla':6, 'Morelos':6, 'Querétaro':7, 'Yucatán':8, 'Chiapas':9, 'San luis Potosí':9, 'Veracruz':10, 'Guanajuato':11, 'Chihuahua':12, 'Sinaloa':13, 'Tamaulipas':14, 'Oaxaca':14, 'Campeche':14, 'Colima':15, 'Coahuila':15, 'Sonora':16, 'Michoacán':16, 'Aguascalientes':16, 'Zacatecas':17, 'Nayarit':18, 'Hidalgo':19, 'Baja California Sur':20, 'Durango':20}
def top_provincias(x):
    if x in prov_por_precio:
        return prov_por_precio[x]
    else:
        return np.nan

# Intervalos metros
punto20 = df_train['metrostotales'].quantile(0.2)
punto40 = df_train['metrostotales'].quantile(0.4)
punto60 = df_train['metrostotales'].quantile(0.6)
punto80 = df_train['metrostotales'].quantile(0.8)
punto20_cubiertos = df_train['metroscubiertos'].quantile(0.2)
punto40_cubiertos = df_train['metroscubiertos'].quantile(0.4)
punto60_cubiertos = df_train['metroscubiertos'].quantile(0.6)
punto80_cubiertos = df_train['metroscubiertos'].quantile(0.8)

# Division de tipo de propiedad
propiedades_compartidas = ['Casa en condominio', 'Duplex', 'Departamento Compartido','Lote']
propiedades_frecuentes = ['Casa', 'Apartamento', 'Edificio']

# Escuelas y Centros Comerciales Cercanos
def cercanos(x):
    if not x['escuelascercanas'] and not x['centroscomercialescercanos']:
        return 'Ninguno'
    if x['escuelascercanas'] and not x['centroscomercialescercanos']:
        return 'Centro Comerciales'
    if not x['centroscomercialescercanos'] and x['escuelascercanas']:
        return 'Escuelas Cercanas'
    return 'Ambos'

# Delincuencia
ciudades_mas_peligrosas = pd.read_csv('./data/ciudades_mas_peligrosas.csv')
df_crimenes = pd.merge(df_train, ciudades_mas_peligrosas, on = 'ciudad', how= 'left')

# Turismo
ciudades_mas_turisticas = pd.read_csv('./data/ciudades_mas_turisticas.csv')
df_turismo = pd.merge(df_train, ciudades_mas_turisticas, on = 'ciudad', how= 'left')

# Personas por propiedad
def calculo_inquilinos(x):
    if (x['habitaciones'] == np.nan) or (x['banos'] == np.nan) or (x['habitaciones'] == 0) or (x['banos'] == 0):
        return 0
    if (x['habitaciones'] == 1) and (x['banos'] == 1):
        return 1
    if (x['habitaciones'] == 2) and (x['banos'] == 1):
        return 3
    if (x['habitaciones'] == 2) and (x['banos'] == 2):
        return 4
    if (x['habitaciones'] == 3) and (x['banos'] == 1):
        return 5
    if (x['habitaciones'] == 3) and (x['banos'] == 2):
        return 6
    if (x['habitaciones'] == 4) and (x['banos'] == 1):
        return 6
    if (x['habitaciones'] == 4) and (x['banos'] == 2):
        return 7
    if (x['habitaciones'] == 4) and ((x['banos'] == 3) or (x['banos'] == 4)):
        return 8
    if (x['habitaciones'] == 5):
        return 10
    if (x['habitaciones'] > 5):
        return 11

# Ciudad Centrica por provincia
ciudades_centricas = pd.read_csv('./data/ciudadcentrica_por_provincia.csv')
df_centro = pd.merge(df_train, ciudades_centricas, on = 'ciudad', how= 'left')

In [57]:
def features_independientes_precio(df):
    '''Devuelve una copia del dataframe con nuevas columnas, 
    donde los datos generados no dependen del precio'''
    df = df.copy()
    
    # Por Tipo de propiedad
    df['escomercial'] = df.tipodepropiedad.str.contains('.*omercial.*').astype(bool)
    df['promedio_metros_tipo_propiedad'] = df.groupby(['tipodepropiedad'])['metrostotales'].transform('mean')
    df['promedio_metros_tipo_propiedad'] = df.groupby(['tipodepropiedad'])['metroscubiertos'].transform('mean')
    df['tipo_propiedad_compartida'] = list(map(lambda x: True if x in propiedades_compartidas else False, df['tipodepropiedad']))
    df['prop_frecuente'] = list(map(lambda x: True if x in propiedades_frecuentes else False, df['tipodepropiedad']))     

    # Por ubicación
    df['zona'] = list(map(lambda x: 'Norte ' if x in provincias_norte else ('Sur' if x in provincias_sur else 'Centro'), df['provincia']))
    df['top_provincia'] = df_train['provincia'].apply(top_provincias)
    df['es_ciudad_centrica'] = df_centro['centro'].replace({np.nan:False})
    df['promedio_metros_totales_provincia'] = df_train.groupby(['provincia'])['metrostotales'].transform('mean')
    df['promedio_metros_cubiertos_provincia'] = df_train.groupby(['provincia'])['metroscubiertos'].transform('mean')
    
    # Por metros totales  cubiertos
    df['porcentaje_metros'] = df['metroscubiertos']/df['metrostotales']
    df['diferencia_metros'] = df['metrostotales'] - df['metroscubiertos']
    df['intervalo_metros_totales'] = list(map(lambda x: 'Grupo1' if x<punto20 else ('Grupo2' if punto20<x<punto40 else ('Grupo3' if punto40<x<punto60 else('Grupo4' if punto60<x<punto80 else 'Grupo5'))), df['metrostotales']))
    df['intervalo_metros_cubiertos'] = list(map(lambda x: 'Grupo1' if x<punto20_cubiertos else ('Grupo2' if punto20_cubiertos<x<punto40_cubiertos else ('Grupo3' if punto40_cubiertos<x<punto60_cubiertos else('Grupo4' if punto60_cubiertos<x<punto80_cubiertos else 'Grupo5'))), df['metrostotales']))

    # Por fecha
    df['fecha'] = pd.to_datetime(df_train['fecha'])
    df['anio'] = df['fecha'].dt.year
    df['mes'] = df['fecha'].dt.month
    df['dia'] = df['fecha'].dt.day
    df['trimestre'] = (df['fecha'].dt.month)%3+1
    
    # Propiedades booleanas
    df['escualas_centros_cercanos'] = df_train[['centroscomercialescercanos', 'escuelascercanas']].apply(cercanos, axis=1)
    df['delincuencia'] = df_crimenes['peligro'].replace({np.nan:False})
    df['turismo'] = df_turismo['turismo'].replace({np.nan:False})
    df['es_antigua'] = list(map(lambda x: True if x>30 else False, df_train['antiguedad']))

    # Habitaciones, garages y banos
    df['cantidad_inquilinos'] = df_train[['habitaciones', 'banos']].apply(calculo_inquilinos,axis=1)
    
    return df

In [74]:
def features_dependientes_precio(df, df_train):
    '''Devuelve una copia del dataframe `df` con nuevas columnas dependientes del precio,
    usando el dataframe `df_train` con los precios para generar los datos.
    El dataframe `df` puede ser el mismo que `df_train`.'''
    df = df.copy()
    df_train = df_train.copy()
    
    # Por ciudad
    df_train['promedio_precio_ciudad'] = df_train.groupby(['provincia', 'ciudad'])['precio'].transform('mean')
    df_promedios = df_train.groupby(['provincia', 'ciudad']).agg({'precio':'mean'}).rename(columns={'precio':'promedio_precio_ciudad'}).reset_index()
    df = pd.merge(df, df_promedios, on=['provincia', 'ciudad'], how='left')
    df['promedio_precio_ciudad'] = df['promedio_precio_ciudad'].fillna(df_train['promedio_precio_ciudad'].mean())
    
    df_train['varianza_precio_ciudad'] = df_train.groupby(['provincia', 'ciudad'])['precio'].transform('std')
    df_varianzas = df_train.groupby(['provincia', 'ciudad']).agg({'precio':'std'}).rename(columns={'precio':'varianza_precio_ciudad'}).reset_index()
    df = pd.merge(df, df_varianzas, on=['provincia', 'ciudad'], how='left')
    df['varianza_precio_ciudad'] = df['varianza_precio_ciudad'].fillna(df_train['varianza_precio_ciudad'].std())

    # Por id_zona
    df_train['promedio_id_zona'] = df_train.groupby(['idzona'])['precio'].transform('mean')
    df_promedio = df_train.groupby('idzona').agg({'precio':'mean'}).rename(columns = {'precio': 'promedio_id_zona'})
    df = pd.merge(df, df_promedio, on='idzona', how='left')
    df['promedio_id_zona'] = df['promedio_id_zona'].fillna(df['promedio_precio_ciudad'])

    df_train['varianza_id_zona'] = df_train.groupby(['idzona'])['precio'].transform('std')
    df_varianza = df_train.groupby('idzona').agg({'precio':'std'}).rename(columns = {'precio': 'varianza_id_zona'})
    df = pd.merge(df, df_varianza, on='idzona', how='left')
    df['varianza_id_zona'] = df['varianza_id_zona'].fillna(df['varianza_precio_ciudad'])

    # Por tipo de propiedad
    df_train['tipodepropiedad'] = df_train['tipodepropiedad'].fillna('otro') # Otra forma de llenar los nans para el tipo de prop?
    df['promedio_precio_tipo_propiedad'] = df_train.groupby(['tipodepropiedad'])['precio'].transform('mean')
       
    # Por fecha
    df_train['fecha'] = pd.to_datetime(df_train['fecha'])
    df_train['anio'] = df_train['fecha'].dt.year
    df_train['mes'] = df_train['fecha'].dt.month
    df_train['dia'] = df_train['fecha'].dt.day
    df['promedio_por_mes'] = df_train.groupby(['anio','mes'])['precio'].transform('mean')
    df['varianza_por_mes'] = df_train.groupby(['anio','mes'])['precio'].transform('std')

    # Por cantidad de habitaciones, banos y garages
    df_train['promedio_precio_habitaciones'] = df_train.groupby(['habitaciones'])['precio'].transform('mean')
    df_train['promedio_precio_habitaciones_banos_garages'] = df_train.groupby(['habitaciones', 'banos', 'garages'])['precio'].transform('mean')
    df_train['promedio_precio_banos_garages'] = df_train.groupby(['banos', 'garages'])['precio'].transform('mean')
    df_habitaciones = df_train.groupby(['tipodepropiedad', 'banos', 'habitaciones', 'garages']).agg({'precio': 'mean'}).rename(columns = {'precio': 'promedio_precio_habitaciones'}).reset_index()
    df = pd.merge(df, df_habitaciones, on=['tipodepropiedad', 'banos', 'habitaciones', 'garages'], how = 'left')
    df['promedio_precio_habitaciones'] = df['promedio_precio_habitaciones'].fillna(df_train['promedio_precio_habitaciones'].mean())
    df['promedio_precio_habitaciones_banos_garages'] = df_train['promedio_precio_habitaciones_banos_garages'].fillna(df_train['promedio_precio_habitaciones'].mean())
    df['promedio_precio_banos_garages'] = df_train['promedio_precio_banos_garages'].fillna(df_train['promedio_precio_habitaciones'].mean())
        
    # Por propiedades booleanas
    df['promedio_precio_booleanos'] = df_train.groupby(['gimnasio', 'usosmultiples', 'piscina'])['precio'].transform('mean')
        
    # Insight    
    df['puntaje'] = zona['puntaje']
    
    return df

In [75]:
# Garages
zonaPromedioGarages = df_train.groupby(['tipodepropiedad', 'habitaciones']).agg({'garages':'mean'}).reset_index()
df_train['garages'] = df_train['garages'].fillna(zonaPromedioGarages['garages'])

promedioGarages = zonaPromedioGarages['garages'].mean()
df_train['garages'] = df_train['garages'].fillna(promedioGarages)

# Habitaciones
zonaPromedioHabitaciones = df_train.groupby(['tipodepropiedad', 'provincia']).agg({'habitaciones':'mean'}).reset_index()
df_train['habitaciones'] = df_train['habitaciones'].fillna(zonaPromedioHabitaciones['habitaciones'])

promedioHabitaciones = zonaPromedioHabitaciones['habitaciones'].mean()
df_train['habitaciones'] = df_train['habitaciones'].fillna(promedioHabitaciones)

# Banos
zonaPromedioBanos = df_train.groupby(['tipodepropiedad', 'provincia']).agg({'banos':'mean'}).reset_index()
df_train['banos'] = df_train['banos'].fillna(zonaPromedioBanos['banos'])

promedioBanos = zonaPromedioBanos['banos'].mean()
df_train['banos'] = df_train['banos'].fillna(promedioBanos)

In [76]:
# Insight
zona = df_train
punto2 = zona['precio'].quantile(0.2)
punto4 = zona['precio'].quantile(0.4)
punto6 = zona['precio'].quantile(0.6)
punto8 = zona['precio'].quantile(0.8)

In [77]:
zona_ciudad = zona.groupby('ciudad').agg('mean')
zona_metros = zona.groupby('metroscubiertos').agg('mean')
zona_metros_cub = zona.groupby('metrostotales').agg('mean')


zona_ciudad['grupoCiudad'] = list(map(lambda x: 'Grupo 1' if x < punto2 else ('Grupo 2' if punto2<x<punto4 else ('Grupo 3' if punto4<x<punto6 else ('Grupo 4' if punto6<x<punto8 else 'Grupo 5'))), zona_ciudad["precio"]))
zona_ciudad_un= zona_ciudad.unstack()

zona_metros['grupoMetros'] = list(map(lambda x: 'Grupo 1' if x < punto2 else ('Grupo 2' if punto2<x<punto4 else ('Grupo 3' if punto4<x<punto6 else ('Grupo 4' if punto6<x<punto8 else 'Grupo 5'))), zona_metros["precio"]))
zona_metros_un = zona_metros.unstack()

zona_metros_cub['grupoCubiertos'] = list(map(lambda x: 'Grupo 1' if x < punto2 else ('Grupo 2' if punto2<x<punto4 else ('Grupo 3' if punto4<x<punto6 else ('Grupo 4' if punto6<x<punto8 else 'Grupo 5'))), zona_metros_cub["precio"]))
zona_metros_cub_un = zona_metros_cub.unstack()

In [78]:
provincias1 = ['Guerrero', 'Tabasco', 'Nueva León', 'Quintana Roo', 'Morelos', 'Jalisco', 'Puebla']
provincias2 = ['Querétaro', 'Yucatán', 'Chiapas', 'Guanajuato', 'San Luis Potosí', 'Veracruz', 'Chihuahua', 'Sinaloa']
provincias3 = ['Tamaulipas', 'Oaxaca', 'Campeche', 'Campeche', 'Zacatecas', 'Colima', 'Aguascalientes', 'Michoacán', 'Sonora', 'Nayarit'] 

def df_puntajes(zona):
    for i in range(len(zona)):
        puntaje = 0
        if zona.piscina[i] == 1.0:
            puntaje += 8
        if zona.usosmultiples[i] == 1.0:
            puntaje +=5
        if zona.gimnasio[i] == 1.0:
            puntaje +=5
        if zona.banos[i] == 2.0:
            puntaje +=2
        elif zona.banos[i] == 3.0:
            puntaje += 5
        elif zona.banos[i] == 4.0:
            puntaje += 6
        if zona.habitaciones[i] == 2.0:
            puntaje += 2
        elif zona.habitaciones[i] == 3.0:
            puntaje += 5
        elif zona.habitaciones[i] > 3.0:
            puntaje += 6
        if (zona.ciudad[i] not in zona_ciudad.index):
            puntaje += 0
        elif zona_ciudad_un[('grupoCiudad',zona.ciudad[i])] == 'Grupo 2':
            puntaje += 2
        elif zona_ciudad_un[('grupoCiudad',zona.ciudad[i])] == 'Grupo 3':
            puntaje += 5
        elif zona_ciudad_un[('grupoCiudad',zona.ciudad[i])] == 'Grupo 4':
            puntaje += 8
        elif zona_ciudad_un[('grupoCiudad',zona.ciudad[i])] == 'Grupo 5':
            puntaje += 10
        if (zona.metrostotales[i] not in zona_metros.index) :
            puntaje += 0
        elif zona_metros_un[('grupoMetros',zona.metrostotales[i])] == 'Grupo 2':
            puntaje += 4
        elif zona_metros_un[('grupoMetros',zona.metrostotales[i])] == 'Grupo 3':
            puntaje += 6
        elif zona_metros_un[('grupoMetros',zona.metrostotales[i])] == 'Grupo 4':
            puntaje += 8
        elif zona_metros_un[('grupoMetros',zona.metrostotales[i])] == 'Grupo 5':
            puntaje += 10
        if (zona.metroscubiertos[i] not in zona_metros_cub.index):
            puntaje +=0
        elif zona_metros_cub_un[('grupoCubiertos',zona.metroscubiertos[i])] == 'Grupo 2':
            puntaje += 4
        elif zona_metros_cub_un[('grupoCubiertos',zona.metroscubiertos[i])] == 'Grupo 3':
            puntaje += 6
        elif zona_metros_cub_un[('grupoCubiertos',zona.metroscubiertos[i])] == 'Grupo 4':
            puntaje += 8
        elif zona_metros_cub_un[('grupoCubiertos',zona.metroscubiertos[i])] == 'Grupo 5':
            puntaje += 10
        if zona.provincia[i] == 'Distrito Federal':
            puntaje += 8
        elif zona.provincia[i] == 'Edo. de México':
            puntaje += 7
        elif zona.provincia[i] in provincias1:
            puntaje += 5
        elif zona.provincia[i] in provincias2:
            puntaje += 4
        elif zona.provincia[i] in provincias3:
            puntaje += 2
        zona['puntaje'][i] = puntaje

In [63]:
zona['puntaje']=0
imp = SimpleImputer()
zona['metroscubiertos'] = imp.fit_transform(zona[['metroscubiertos']])
zona['metrostotales'] = imp.fit_transform(zona[['metrostotales']])
zona = zona.reset_index()

In [90]:
# Tarda un buen rato
df_puntajes(zona)

/home/camix/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:66: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [94]:
ind = features_independientes_precio(df_train)

In [95]:
ind[['escomercial',
       'promedio_metros_tipo_propiedad', 'tipo_propiedad_compartida',
       'prop_frecuente', 'zona','top_provincia', 'es_ciudad_centrica',
       'promedio_metros_totales_provincia',
       'promedio_metros_cubiertos_provincia', 'porcentaje_metros',
       'diferencia_metros', 'intervalo_metros_totales',
       'intervalo_metros_cubiertos', 'anio', 'mes', 'dia', 'trimestre',
       'escualas_centros_cercanos', 'delincuencia', 'turismo', 'es_antigua', 'cantidad_inquilinos']].head()

,escomercial,promedio_metros_tipo_propiedad,tipo_propiedad_compartida,prop_frecuente,zona,top_provincia,es_ciudad_centrica,promedio_metros_totales_provincia,promedio_metros_cubiertos_provincia,porcentaje_metros,...,intervalo_metros_cubiertos,anio,mes,dia,trimestre,escualas_centros_cercanos,delincuencia,turismo,es_antigua,cantidad_inquilinos
0,False,117.866305,False,True,Centro,0.0,True,166.863407,157.327271,1.000000,...,Grupo5,2015,8,23,3,Ninguno,False,False,False,4.0
1,False,187.276828,True,False,Centro,0.0,False,166.863407,157.327271,1.488889,...,Grupo3,2013,6,28,1,Ambos,False,False,False,6.0
2,False,193.104607,False,True,Centro,5.0,False,165.593502,171.954157,0.867470,...,Grupo3,2015,10,17,2,Ninguno,False,False,False,6.0
3,False,193.104607,False,True,Centro,1.0,False,174.069659,179.410672,0.940299,...,Grupo1,2012,3,9,1,Ambos,False,False,False,3.0
4,False,117.866305,False,True,Centro,5.0,True,165.593502,171.954157,1.000000,...,Grupo2,2016,6,7,1,Ninguno,False,False,False,3.0


In [96]:
dep = features_dependientes_precio(df_train, df_train)

In [99]:
dep[['precio','promedio_precio_ciudad',
       'varianza_precio_ciudad', 'promedio_id_zona', 'varianza_id_zona',
       'promedio_precio_tipo_propiedad', 'promedio_por_mes',
       'varianza_por_mes', 'promedio_precio_habitaciones',
       'promedio_precio_habitaciones_banos_garages',
       'promedio_precio_banos_garages', 'promedio_precio_booleanos', 'puntaje']].head()

,precio,promedio_precio_ciudad,varianza_precio_ciudad,promedio_id_zona,varianza_id_zona,promedio_precio_tipo_propiedad,promedio_por_mes,varianza_por_mes,promedio_precio_habitaciones,promedio_precio_habitaciones_banos_garages,promedio_precio_banos_garages,promedio_precio_booleanos,puntaje
0,2273000.0,3.384934e+06,2.096613e+06,3.036895e+06,1.604525e+06,2.763769e+06,2.654494e+06,2.195410e+06,2.299896e+06,1.943985e+06,1.742841e+06,2.398349e+06,28
1,3600000.0,4.605869e+06,2.510121e+06,3.430078e+06,1.869285e+06,2.898927e+06,2.058968e+06,1.813118e+06,2.127411e+06,2.196206e+06,2.399175e+06,2.398349e+06,41
2,1200000.0,8.695932e+05,6.776495e+05,1.040921e+06,2.276313e+05,2.398158e+06,2.712010e+06,2.200963e+06,1.939210e+06,2.196206e+06,2.399175e+06,2.398349e+06,28
3,650000.0,1.472305e+06,9.851286e+05,1.545000e+06,7.637277e+05,2.398158e+06,1.966355e+06,1.842065e+06,5.953745e+05,8.014207e+05,9.232776e+05,2.398349e+06,19
4,1150000.0,2.749675e+06,2.121691e+06,1.938494e+06,1.218248e+06,2.763769e+06,2.744906e+06,2.252515e+06,1.026163e+06,8.014207e+05,9.232776e+05,2.398349e+06,27


In [98]:
dep.isnull().any()

id                                            False
titulo                                         True
descripcion                                    True
tipodepropiedad                                True
direccion                                      True
ciudad                                         True
provincia                                      True
antiguedad                                     True
habitaciones                                  False
garages                                       False
banos                                         False
metroscubiertos                               False
metrostotales                                 False
idzona                                         True
lat                                            True
lng                                            True
fecha                                         False
gimnasio                                      False
usosmultiples                                 False
piscina     